In [2]:
!pip install unidecode numpy cython

     |████████████████████████████████| 245kB 2.8MB/s 


In [3]:
!pip install -U biogeme

     |████████████████████████████████| 1.2MB 2.8MB/s 
  Created wheel for biogeme: filename=biogeme-3.2.5-cp36-cp36m-linux_x86_64.whl size=3244383 sha256=cdaee61b4655aadfd0905fd3ef24e480f07f9b8e027db545b92e374812260f85
  Stored in directory: /root/.cache/pip/wheels/04/c4/65/792136fc4a56332020a8c1cab9f606a03119c76b549a0d6741
Successfully built biogeme


In [0]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable, Variable
import biogeme.results as res
import biogeme.messaging as msg
import numpy as np
import sys
import os
from scipy.stats import chi2
from sklearn.model_selection import train_test_split

In [0]:
## Read the data
df = pd.read_table("http://transp-or.epfl.ch/data/swissmetro.dat")

##Change variable 'PURPOSE'
#PRIVATE (1) - Shopping, Return from shopping, Leisure, return from leisure
#COMMUTER (2) - Commuter, return from work
#BUSINESS (3) - Business, return from business
df.loc[:, ('PURPOSE')][(df['PURPOSE']==1)] = 2
df.loc[:, ('PURPOSE')][(df['PURPOSE']==2)] = 1
df.loc[:, ('PURPOSE')][(df['PURPOSE']==3)] = 3
df.loc[:, ('PURPOSE')][(df['PURPOSE']==4)] = 1
df.loc[:, ('PURPOSE')][(df['PURPOSE']==5)] = 2
df.loc[:, ('PURPOSE')][(df['PURPOSE']==6)] = 1
df.loc[:, ('PURPOSE')][(df['PURPOSE']==7)] = 3
df.loc[:, ('PURPOSE')][(df['PURPOSE']==8)] = 1

##Remove variables
df = df[(df['AGE'] !=6 ) & (df['PURPOSE'] !=9 ) & (df['CHOICE'] !=0 )]

##Write to biogeme database
database = db.Database("swissmetro",df)

##The following statement allows you to use the names of the variable as Python variable.
globals().update(database.variables)
    

# Miscelanious

In [0]:
###Parameters to be estimated (Note not all parameters are used in all models!)
##Attributes
#Alternative specific constants
ASC_CAR        = Beta('ASC_CAR',0,None,None,1)
ASC_TRAIN      = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM         = Beta('ASC_SM',0,None,None,0)

#Cost (Note: Assumed generic)
B_COST          = Beta('B_COST',0,None,None,0)
B_COST_BUSINESS = Beta('B_COST_BUSINESS',0,None,None,0)
B_COST_PRIVATE  = Beta('B_COST_PRIVATE',0,None,None,0)

#Time
B_TIME                = Beta('B_TIME',0,None,None,0)
B_TIME_CAR            = Beta('B_TIME_CAR',0,None,None,0)
B_TIME_TRAIN          = Beta('B_TIME_TRAIN',0,None,None,0)
B_TIME_SM             = Beta('B_TIME_SM',0,None,None,0)
B_TIME_PUB            = Beta('B_TIME_PUB',0,None,None,0)
B_TIME_CAR_BUSINESS   = Beta('B_TIME_CAR_BUSINESS',0,None,None,0)
B_TIME_TRAIN_BUSINESS = Beta('B_TIME_TRAIN_BUSINESS',0,None,None,0)
B_TIME_SM_BUSINESS    = Beta('B_TIME_SM_BUSINESS',0,None,None,0)
B_TIME_PUB_BUSINESS   = Beta('B_TIME_PUB_BUSINESS',0,None,None,0)
B_TIME_CAR_PRIVATE    = Beta('B_TIME_CAR_PRIVATE',0,None,None,0)
B_TIME_TRAIN_PRIVATE  = Beta('B_TIME_TRAIN_PRIVATE',0,None,None,0)
B_TIME_SM_PRIVATE     = Beta('B_TIME_SM_PRIVATE',0,None,None,0)
B_TIME_PUB_PRIVATE    = Beta('B_TIME_PUB_PRIVATE',0,None,None,0)

#HE (Note: Not available for car)
B_HE                = Beta('B_HE',0,None,None,0)
B_HE_TRAIN          = Beta('B_HE_TRAIN',0,None,None,0)
B_HE_SM             = Beta('B_HE_SM',0,None,None,0)
B_HE_BUSINESS       = Beta('B_HE_BUSINESS',0,None,None,0)
B_HE_TRAIN_BUSINESS = Beta('B_HE_TRAIN_BUSINESS',0,None,None,0)
B_HE_SM_BUSINESS    = Beta('B_HE_SM_BUSINESS',0,None,None,0)
B_HE_PRIVATE        = Beta('B_HE_PRIVATE',0,None,None,0)
B_HE_TRAIN_PRIVATE  = Beta('B_HE_TRAIN_PRIVATE',0,None,None,0)
B_HE_SM_PRIVATE     = Beta('B_HE_SM_PRIVATE',0,None,None,0)

#Seats (Note: Only avaliable for SM)
B_SEATS = Beta('B_SEATS',0,None,None,0)

##Characteristics
#Age
B_AGE_1_TRAIN      = Beta('B_AGE_1_TRAIN',0,None,None,1) #Note: Reference
B_AGE_2_TRAIN      = Beta('B_AGE_2_TRAIN',0,None,None,0)
B_AGE_3_TRAIN      = Beta('B_AGE_3_TRAIN',0,None,None,0)
B_AGE_4_TRAIN      = Beta('B_AGE_4_TRAIN',0,None,None,0)
B_AGE_5_TRAIN      = Beta('B_AGE_5_TRAIN',0,None,None,0)
B_AGE_6_TRAIN      = Beta('B_AGE_6_TRAIN',0,None,None,0)
B_AGE_1_SM         = Beta('B_AGE_1_SM',0,None,None,1) #Note: Reference
B_AGE_2_SM         = Beta('B_AGE_2_SM',0,None,None,0)
B_AGE_3_SM         = Beta('B_AGE_3_SM',0,None,None,0)
B_AGE_4_SM         = Beta('B_AGE_4_SM',0,None,None,0)
B_AGE_5_SM         = Beta('B_AGE_5_SM',0,None,None,0)
B_AGE_6_SM         = Beta('B_AGE_6_SM',0,None,None,0)
B_AGE_1_PUB        = Beta('B_AGE_1_PUB',0,None,None,1) #Note: Reference
B_AGE_2_PUB        = Beta('B_AGE_2_PUB',0,None,None,0)
B_AGE_3_PUB        = Beta('B_AGE_3_PUB',0,None,None,0)
B_AGE_4_PUB        = Beta('B_AGE_4_PUB',0,None,None,0)
B_AGE_5_PUB        = Beta('B_AGE_5_PUB',0,None,None,0)
B_AGE_6_PUB        = Beta('B_AGE_6_PUB',0,None,None,0)
B_AGE_ADULTS_TRAIN = Beta('B_AGE_TRAIN_ADULTS',0,None,None,0)
B_AGE_ADULTS_SM    = Beta('B_AGE_ADULTS_SM',0,None,None,0)
B_AGE_ADULTS_PUB   = Beta('B_AGE_ADULTS_PUB',0,None,None,0)

#Luggage
B_LUGGAGE_TRAIN = Beta('B_LUGGAGE_TRAIN', 0, None, None, 0)
B_LUGGAGE_SM    = Beta('B_LUGGAGE_SM', 0, None, None, 0)
B_LUGGAGE_PUB   = Beta('B_LUGGAGE_PUB', 0, None, None, 0)

#Gender
B_MALE_TRAIN = Beta('B_MALE_TRAIN',0,None,None,0)
B_MALE_SM    = Beta('B_MALE_SM',0,None,None,0)
B_MALE_PUB   = Beta('B_MALE_PUB',0,None,None,0)

#Purpose
B_BUSINESS       = Beta('B_BUSINESS',0,None,None,0)
B_BUSINESS_TRAIN = Beta('B_BUSINESS_TRAIN',0,None,None,0)
B_BUSINESS_SM    = Beta('B_BUSINESS_SM',0,None,None,0)
B_PRIVATE        = Beta('B_PRIVATE',0,None,None,0)
B_PRIVATE_TRAIN  = Beta('B_PRIVATE_TRAIN',0,None,None,0)
B_PRIVATE_SM     = Beta('B_PRIVATE_SM',0,None,None,0)
B_COMMUTER       = Beta('B_COMMUTER',0,None,None,0)
B_COMMUTER_TRAIN = Beta('B_COMMUTER_TRAIN',0,None,None,0)
B_COMMUTER_SM    = Beta('B_COMMUTER_SM',0,None,None,0)

#GA
B_GA        = Beta('B_GA',0,None,None,0)
B_GA_TRAIN  = Beta('B_GA_TRAIN',0,None,None,0)
B_GA_SM     = Beta('B_GA_SM',0,None,None,0)

#First
B_FIRST_TRAIN = Beta('B_FIRST_TRAIN',0,None,None,0)
B_FIRST_SM    = Beta('B_FIRST_SM',0,None,None,0)
B_FIRST       = Beta('B_FIRST',0,None,None,0)

##Non linearization
#Cost
q_COST = Beta('q_COST_CAR',1,None,None,0)

#Time
q_TIME       = Beta('q_TIME',1,None,None,0)
q_TIME_TRAIN = Beta('q_TIME_TRAIN',1,None,None,0)
q_TIME_SM    = Beta('q_TIME_SM',1,None,None,0)
q_TIME_CAR   = Beta('q_TIME_CAR',1,None,None,0)
q_TIME_PUB   = Beta('q_TIME_PUB',1,None,None,0)

#HE
q_HE = Beta('q_HE',1,None,None,0)

##Nesting parameter
MU = Beta('MU',1,0,1,0)

In [0]:
###Manipulating variables
## Setting the cost to 0 for those with annual pass 
SM_COST    =  SM_CO   * (  GA   ==  0  )   
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

##Scaling 'COST', 'TRAVEL-TIME' and 'HE' by a factor of 100 and adding the scaled variables to the database
CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)
 
TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
TRAIN_HE_SCALED = DefineVariable('TRAIN_HE_SCALED',\
                                   TRAIN_HE / 100, database)

SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
SM_HE_SCALED = DefineVariable('SM_HE_SCALED',\
                                   SM_HE / 100, database)

CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_COST_SCALED = DefineVariable('CAR_COST_SCALED', CAR_CO / 100,database)

In [0]:
###Defining new variables and adding columns to the database
#Age
AGE_1 = DefineVariable('AGE_1', (  AGE   ==  1  ),database) #don't scale because is cathegorical
AGE_2 = DefineVariable('AGE_2', (  AGE   ==  2  ),database)
AGE_3 = DefineVariable('AGE_3', (  AGE   ==  3  ),database)
AGE_4 = DefineVariable('AGE_4', (  AGE   ==  4  ),database)
AGE_5 = DefineVariable('AGE_5', (  AGE   ==  5  ),database)
AGE_6 = DefineVariable('AGE_6', (  AGE   ==  6  ),database)

#Purpose
PRIVATE  = DefineVariable("PRIVATE", (PURPOSE == 1), database)
COMMUTER = DefineVariable("COMMUTER", (PURPOSE == 2), database)
BUSINESS = DefineVariable("BUSINESS", (PURPOSE == 3), database)

In [0]:
def estimate_MNL(v1, v2, v3, model_name, modeltype="MNL", nest1=[1,3], nest2=[2]):
    '''
    Function that based on Utility function estimates a MNL model (or NL model) 
    and outputs the LogLikelihood and number of parameters'''
    ## Associate utility functions with the numbering of alternatives
    V = {1: V1,
         2: V2,
         3: V3}

    ## Associate the availability conditions with the alternatives
    av = {1: TRAIN_AV_SP,
          2: SM_AV,
          3: CAR_AV_SP}
    

    
    ## observation to the log likelihood function.
    if modeltype == 'MNL':
        logprob = models.loglogit(V,av,CHOICE)
    else:
        Nest1 = 1, nest1
        Nest2 = 1, nest2
        nests = Nest1, Nest2
    
        logprob = models.lognestedMevMu(V,av,nests,CHOICE,MU)
    
    #Define level of verbosity
    logger = msg.bioMessage()
    logger.setDebug()
    
    ## Create the Biogeme object
    biogeme  = bio.BIOGEME(database,logprob,numberOfThreads=None, skipAudit = True)
    biogeme.modelName = model_name

    ## Estimate the parameters
    results = biogeme.estimate(saveIterations=False)
    biogeme.createLogFile()

    # Get the results in a pandas table
    pandasResults = results.getEstimatedParameters()
    print(pandasResults)
    
    return results.data.logLike, results.data.nparam, results.data.betaNames, results.data.betaValues;

In [0]:
def likelihoodRatioTest(ll1, nparam1, model_name1, ll2, nparam2, model_name2, prob = 0.95):
    '''
    Function that based on two models (Note: One must be a restriction of the other) uses the 
    Likelihood Ratio test to determine if we can reject the null-hypothesis that the restricted
    model describes the data equally well.
    '''
    dof = abs(nparam2 - nparam1)
    LR = -2*(ll1 - ll2)
    critical = chi2.ppf(prob, dof)
    
    if abs(LR) >= critical:
        print('LR = %.2f, Chi2 = %.2f (%.0f defrees of freedom) \nTherefore: Accept more params' %(abs(LR), critical, dof))
    else:
        print('LR = %.2f, Chi2 = %.2f (%.0f defrees of freedom) \nTherefore: Do not accept more params' %(abs(LR), critical, dof))

# MNL (LPLA)

In [0]:
%%capture
## Model 1
#ASC ONLY
model_name = "MNL1"

#Utility functions
V1 = ASC_TRAIN

V2 = ASC_SM 

V3 = ASC_CAR   

#Run MNL
LL1, nparam1, estimate_names1, estimate_values1 = estimate_MNL(V1, V2, V3, model_name)

In [0]:
%%capture
## Model 2
#Generic cost
model_name = "MNL2"

#Utility functions
V1 = ASC_TRAIN + \
     B_COST * TRAIN_COST_SCALED

V2 = ASC_SM + \
     B_COST * SM_COST_SCALED

V3 = ASC_CAR + \
     B_COST * CAR_COST_SCALED

#Run MNL
LL2, nparam2, estimate_names2, estimate_values2 = estimate_MNL(V1, V2, V3, model_name)

In [13]:
likelihoodRatioTest(LL1, nparam1, "MNL1", LL2, nparam2, "MNL2", prob = 0.99)

LR = 492.55, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 3
#Generic time
model_name = "MNL3"

#Utility functions
V1 = ASC_TRAIN                  + \
     B_COST * TRAIN_COST_SCALED + \
     B_TIME * TRAIN_TT_SCALED 

V2 = ASC_SM                  + \
     B_COST * SM_COST_SCALED + \
     B_TIME * SM_TT_SCALED

V3 = ASC_CAR                  + \
     B_COST * CAR_COST_SCALED + \
     B_TIME * CAR_TT_SCALED

#Run MNL
LL3, nparam3, estimate_names3, estimate_values3 = estimate_MNL(V1, V2, V3, model_name)

In [15]:
likelihoodRatioTest(LL2, nparam2,'MNL2', LL3, nparam3,'MNL3', prob = 0.99)

LR = 1104.04, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 4
#Generic HE
model_name = "MNL4"

#Utility functions
V1 = ASC_TRAIN                  + \
     B_COST * TRAIN_COST_SCALED + \
     B_TIME * TRAIN_TT_SCALED   + \
     B_HE   * TRAIN_HE_SCALED 

V2 = ASC_SM                  + \
     B_COST * SM_COST_SCALED + \
     B_TIME * SM_TT_SCALED   + \
     B_HE   * SM_HE_SCALED

V3 = ASC_CAR                  + \
     B_COST * CAR_COST_SCALED + \
     B_TIME * CAR_TT_SCALED   

#Run MNL
LL4, nparam4, estimate_names4, estimate_values4 = estimate_MNL(V1, V2, V3, model_name)

In [17]:
likelihoodRatioTest(LL3, nparam3,'MNL3', LL4, nparam4,'MNL4', prob = 0.99)

LR = 61.05, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 5
#Alternative specific time
model_name = "MNL5"

#Utility functions
V1 = ASC_TRAIN                         + \
     B_COST       * TRAIN_COST_SCALED  + \
     B_TIME_TRAIN * TRAIN_TT_SCALED    + \
     B_HE         * TRAIN_HE_SCALED        

V2 = ASC_SM                      + \
     B_COST    * SM_COST_SCALED  + \
     B_TIME_SM * SM_TT_SCALED    + \
     B_HE      * SM_HE_SCALED

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL5, nparam5, estimate_names5, estimate_values5 = estimate_MNL(V1, V2, V3, model_name)

In [19]:
likelihoodRatioTest(LL4, nparam4,'MNL4', LL5, nparam5,'MNL5', prob = 0.99)

LR = 176.76, Chi2 = 9.21 (2 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 6
#Alternative specific time (TRAIN = SM)
model_name = "MNL6"

#Utility functions
V1 = ASC_TRAIN                      + \
     B_COST     * TRAIN_COST_SCALED + \
     B_TIME_PUB * TRAIN_TT_SCALED   + \
     B_HE       * TRAIN_HE_SCALED 

V2 = ASC_SM                      + \
     B_COST     * SM_COST_SCALED + \
     B_TIME_PUB * SM_TT_SCALED   + \
     B_HE       * SM_HE_SCALED

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL6, nparam6, estimate_names6, estimate_values6 = estimate_MNL(V1, V2, V3, model_name)

In [21]:
likelihoodRatioTest(LL5, nparam5,'MNL5', LL6, nparam6,'MNL6', prob = 0.99)

LR = 20.65, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 7
#Alternative specific HE
model_name = "MNL7"

#Utility functions
V1 = ASC_TRAIN                      + \
     B_COST       * TRAIN_COST_SCALED + \
     B_TIME_TRAIN * TRAIN_TT_SCALED   + \
     B_HE_TRAIN   * TRAIN_HE_SCALED 

V2 = ASC_SM                      + \
     B_COST     * SM_COST_SCALED + \
     B_TIME_SM  * SM_TT_SCALED   + \
     B_HE_SM    * SM_HE_SCALED

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL7, nparam7, estimate_names7, estimate_values7 = estimate_MNL(V1, V2, V3, model_name)

In [23]:
likelihoodRatioTest(LL5, nparam5,'MNL5', LL7, nparam7,'MNL7', prob = 0.99)

LR = 0.25, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 8
#Alternative specific age
model_name = "MNL8"

#Utility functions
V1 = ASC_TRAIN                         + \
     B_COST        * TRAIN_COST_SCALED + \
     B_TIME_TRAIN  * TRAIN_TT_SCALED   + \
     B_HE          * TRAIN_HE_SCALED   + \
     B_AGE_2_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN * AGE_3             + \
     B_AGE_4_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN * AGE_5             

V2 = ASC_SM                      + \
     B_COST     * SM_COST_SCALED + \
     B_TIME_SM  * SM_TT_SCALED   + \
     B_HE       * SM_HE_SCALED   + \
     B_AGE_2_SM * AGE_2          + \
     B_AGE_3_SM * AGE_3          + \
     B_AGE_4_SM * AGE_4          + \
     B_AGE_5_SM * AGE_5     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL8, nparam8, estimate_names8, estimate_values8 = estimate_MNL(V1, V2, V3, model_name)

In [25]:
likelihoodRatioTest(LL5, nparam5,'MNL5', LL8, nparam8,'MNL8', prob = 0.99)

LR = 587.63, Chi2 = 20.09 (8 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 9
#Alternative specific age (TRAIN = SM)
model_name = "MNL9"

#Utility functions
V1 = ASC_TRAIN                         + \
     B_COST        * TRAIN_COST_SCALED + \
     B_TIME_TRAIN  * TRAIN_TT_SCALED   + \
     B_HE          * TRAIN_HE_SCALED   + \
     B_AGE_2_PUB   * AGE_2             + \
     B_AGE_3_PUB   * AGE_3             + \
     B_AGE_4_PUB   * AGE_4             + \
     B_AGE_5_PUB   * AGE_5             

V2 = ASC_SM                       + \
     B_COST      * SM_COST_SCALED + \
     B_TIME_SM   * SM_TT_SCALED   + \
     B_HE        * SM_HE_SCALED   + \
     B_AGE_2_PUB * AGE_2          + \
     B_AGE_3_PUB * AGE_3          + \
     B_AGE_4_PUB * AGE_4          + \
     B_AGE_5_PUB * AGE_5     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL9, nparam9, estimate_names9, estimate_values9 = estimate_MNL(V1, V2, V3, model_name)

In [27]:
likelihoodRatioTest(LL8, nparam8,'MNL8', LL9, nparam9,'MNL9', prob = 0.99)

LR = 514.26, Chi2 = 13.28 (4 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 10
#Condense age groups
model_name = "MNL10"

#Utility functions
V1 = ASC_TRAIN                         + \
     B_COST        * TRAIN_COST_SCALED + \
     B_TIME_TRAIN  * TRAIN_TT_SCALED   + \
     B_HE          * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_ADULTS_TRAIN * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN * AGE_5             

V2 = ASC_SM                      + \
     B_COST     * SM_COST_SCALED + \
     B_TIME_SM  * SM_TT_SCALED   + \
     B_HE       * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_ADULTS_SM * AGE_4          + \
     B_AGE_5_SM * AGE_5     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL10, nparam10, estimate_names10, estimate_values10 = estimate_MNL(V1, V2, V3, model_name)

In [29]:
likelihoodRatioTest(LL8, nparam8,'MNL8', LL10, nparam10,'MNL10', prob = 0.99)

LR = 48.81, Chi2 = 13.28 (4 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 11
#Condense agegroups
model_name = "MNL11"

#Utility functions
V1 = ASC_TRAIN                         + \
     B_COST        * TRAIN_COST_SCALED + \
     B_TIME_TRAIN  * TRAIN_TT_SCALED   + \
     B_HE          * TRAIN_HE_SCALED   + \
     B_AGE_2_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN * AGE_3             + \
     B_AGE_4_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN * AGE_5             

V2 = ASC_SM                      + \
     B_COST     * SM_COST_SCALED + \
     B_TIME_SM  * SM_TT_SCALED   + \
     B_HE       * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM * AGE_4          + \
     B_AGE_5_SM * AGE_5     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL11, nparam11, estimate_names11, estimate_values11 = estimate_MNL(V1, V2, V3, model_name)

In [31]:
likelihoodRatioTest(LL8, nparam8,'MNL8', LL11, nparam11,'MNL11', prob = 0.99)

LR = 1.47, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 12
#Condense agegroups
model_name = "MNL12"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL12, nparam12, estimate_names12, estimate_values12 = estimate_MNL(V1, V2, V3, model_name)

In [33]:
likelihoodRatioTest(LL11, nparam11,'MNL11', LL12, nparam12,'MNL12', prob = 0.99)

LR = 0.02, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 13
#Alternative specific luggage
model_name = "MNL13"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL13, nparam13, estimate_names13, estimate_values13 = estimate_MNL(V1, V2, V3, model_name)

In [35]:
likelihoodRatioTest(LL12, nparam12,'MNL12', LL13, nparam13,'MNL13', prob = 0.99)

LR = 78.68, Chi2 = 9.21 (2 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 14
#Alternative specific luggage (TRAIN = SM)
model_name = "MNL14"

#Utility functions
V1 = ASC_TRAIN                         + \
     B_COST        * TRAIN_COST_SCALED + \
     B_TIME_TRAIN  * TRAIN_TT_SCALED   + \
     B_HE          * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN * AGE_5             + \
     B_LUGGAGE_PUB * LUGGAGE

V2 = ASC_SM                      + \
     B_COST     * SM_COST_SCALED + \
     B_TIME_SM  * SM_TT_SCALED   + \
     B_HE       * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM * AGE_4          + \
     B_AGE_5_SM * AGE_5     + \
     B_LUGGAGE_PUB * LUGGAGE

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL14, nparam14, estimate_names14, estimate_values14 = estimate_MNL(V1, V2, V3, model_name)

In [37]:
likelihoodRatioTest(LL13, nparam13,'MNL13', LL14, nparam14,'MNL14', prob = 0.99)

LR = 59.50, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 15
#Remove alternative specific luggage from SM
model_name = "MNL15"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5        

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL15, nparam15, estimate_names15, estimate_values15 = estimate_MNL(V1, V2, V3, model_name)

In [39]:
likelihoodRatioTest(LL13, nparam13,'MNL13', LL15, nparam15,'MNL15', prob = 0.99)

LR = 7.26, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 16
#SM_SEATS
model_name = "MNL16"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_SEATS         * SM_SEATS

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL16, nparam16, estimate_names16, estimate_values16 = estimate_MNL(V1, V2, V3, model_name)

In [41]:
likelihoodRatioTest(LL13, nparam13,'MNL13', LL16, nparam16,'MNL16', prob = 0.99)

LR = 4.56, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 17
#Alternative specific gender
model_name = "MNL17"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_MALE_SM       * MALE

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL17, nparam17, estimate_names17, estimate_values17 = estimate_MNL(V1, V2, V3, model_name)

In [43]:
likelihoodRatioTest(LL13, nparam13,'MNL13', LL17, nparam17,'MNL17', prob = 0.99)

LR = 109.60, Chi2 = 9.21 (2 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 18
#Alternative specific gender (TRAIN = SM)
model_name = "MNL18"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_PUB       * MALE

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_MALE_PUB       * MALE

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL18, nparam18, estimate_names18, estimate_values18 = estimate_MNL(V1, V2, V3, model_name)

In [45]:
likelihoodRatioTest(LL17, nparam17,'MNL17', LL18, nparam18,'MNL18', prob = 0.99)

LR = 90.32, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 19
#Remove alternative specific gender from SM
model_name = "MNL19"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE       

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL19, nparam19, estimate_names19, estimate_values19 = estimate_MNL(V1, V2, V3, model_name)

In [47]:
likelihoodRatioTest(LL17, nparam17,'MNL17', LL19, nparam19,'MNL19', prob = 0.99)

LR = 5.28, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 20
#Alternative specific time (TRAIN = SM)
model_name = "MNL20"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_PUB         * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_PUB       * SM_TT_SCALED  + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE       

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL20, nparam20, estimate_names20, estimate_values20 = estimate_MNL(V1, V2, V3, model_name)

In [49]:
likelihoodRatioTest(LL19, nparam19,'MNL19', LL20, nparam20,'MNL20', prob = 0.99)

LR = 8.61, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 21
#Alternative specific GA
model_name = "MNL21"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_GA_SM         * GA

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL21, nparam21, estimate_names21, estimate_values21 = estimate_MNL(V1, V2, V3, model_name)

In [51]:
likelihoodRatioTest(LL19, nparam19,'MNL19', LL21, nparam21,'MNL21', prob = 0.99)

LR = 429.97, Chi2 = 9.21 (2 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 22
#Remove alternative specific gender from SM
model_name = "MNL22"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA               * GA

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_GA            * GA

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL22, nparam22, estimate_names22, estimate_values22 = estimate_MNL(V1, V2, V3, model_name)

In [53]:
likelihoodRatioTest(LL21, nparam21,'MNL21', LL22, nparam22,'MNL22', prob = 0.99)

LR = 319.14, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 23
#Alternative specific FIRST
model_name = "MNL23"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_FIRST_TRAIN      * FIRST   

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_FIRST_SM      * FIRST

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL23, nparam23, estimate_names23, estimate_values23 = estimate_MNL(V1, V2, V3, model_name)

In [55]:
likelihoodRatioTest(LL21, nparam21,'MNL21', LL23, nparam23,'MNL23', prob = 0.99)

LR = 23.74, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 24
#Alternative specific FIRST
model_name = "MNL24"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_FIRST            * FIRST   

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_FIRST         * FIRST

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL24, nparam24, estimate_names24, estimate_values24 = estimate_MNL(V1, V2, V3, model_name)

In [57]:
likelihoodRatioTest(LL23, nparam23,'MNL23', LL24, nparam24,'MNL24', prob = 0.99)

LR = 12.92, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 25
#Alternative specific PURPOSE
model_name = "MNL25"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_FIRST_TRAIN      * FIRST             + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE_TRAIN    * PRIVATE           + \
     B_COMMUTER_TRAIN   * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE_SM    * PRIVATE        + \
     B_COMMUTER_SM   * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL25, nparam25, estimate_names25, estimate_values25 = estimate_MNL(V1, V2, V3, model_name)

In [59]:
likelihoodRatioTest(LL23, nparam23,'MNL23', LL25, nparam25,'MNL25', prob = 0.99)

LR = 140.04, Chi2 = 16.81 (6 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 26
#Generic PURPOSE
model_name = "MNL26"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_FIRST_TRAIN      * FIRST             + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE          * PRIVATE           + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE       * PRIVATE        + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL26, nparam26, estimate_names26, estimate_values26 = estimate_MNL(V1, V2, V3, model_name)

In [61]:
likelihoodRatioTest(LL25, nparam25,'MNL25', LL26, nparam26,'MNL26', prob = 0.99)

LR = 0.97, Chi2 = 9.21 (2 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 27
#Remove FIRST in train
model_name = "MNL27"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE          * PRIVATE           + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_LUGGAGE_SM    * LUGGAGE        + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE       * PRIVATE        + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL27, nparam27, estimate_names27, estimate_values27 = estimate_MNL(V1, V2, V3, model_name)

In [63]:
likelihoodRatioTest(LL26, nparam26,'MNL26', LL27, nparam27,'MNL27', prob = 0.99)

LR = 0.36, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 28
#Remove luggage in SM
model_name = "MNL28"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE          * PRIVATE           + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE       * PRIVATE        + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL28, nparam28, estimate_names28, estimate_values28 = estimate_MNL(V1, V2, V3, model_name)

In [65]:
likelihoodRatioTest(LL27, nparam27,'MNL27', LL28, nparam28,'MNL28', prob = 0.99)

LR = 6.08, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model 29
#Remove AGE5 in train
model_name = "MNL29"

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE          * PRIVATE           + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE       * PRIVATE        + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LL29, nparam29, estimate_names29, estimate_values29 = estimate_MNL(V1, V2, V3, model_name)

In [67]:
likelihoodRatioTest(LL28, nparam28,'MNL28', LL29, nparam29,'MNL29', prob = 0.99)

LR = 8.51, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


# LPLA Nested Logit

In [0]:
%%capture
## Model NL1
#Based on MNL 28
model_name = "NL1"

nest1 = [1,2] #TRAIN and SM
nest2 = [3]   #CAR

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE          * PRIVATE           + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE       * PRIVATE        + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LLNL1, nparamNL1, estimate_namesNL1, estimate_valuesNL1 = estimate_MNL(V1, V2, V3, model_name, 'NL', nest1, nest2)

In [69]:
likelihoodRatioTest(LL28, nparam28,'MNL28', LLNL1, nparamNL1,'NL1', prob = 0.99)

LR = 0.00, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model NL2
#Based on MNL28
model_name = "NL2"

nest1 = [1,3] #TRAIN and CAR
nest2 = [2]   #SM

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE          * PRIVATE           + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE       * PRIVATE        + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED   

#Run MNL
LLNL2, nparamNL2, estimate_namesNL2, estimate_valuesNL2 = estimate_MNL(V1, V2, V3, model_name, 'NL', nest1, nest2)

In [71]:
likelihoodRatioTest(LL28, nparam28,'MNL28', LLNL2, nparamNL2,'MNLNL2', prob = 0.99)

LR = 51.97, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model NL3
#Based on MNL28
model_name = "NL3"

nest1 = [1]   #TRAIN
nest2 = [2,3] #SM and CAR

#Utility functions
V1 = ASC_TRAIN                              + \
     B_COST             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED   + \
     B_AGE_ADULTS_TRAIN * AGE_2             + \
     B_AGE_3_TRAIN      * AGE_3             + \
     B_AGE_ADULTS_TRAIN * AGE_4             + \
     B_AGE_5_TRAIN      * AGE_5             + \
     B_LUGGAGE_TRAIN    * LUGGAGE           + \
     B_MALE_TRAIN       * MALE              + \
     B_GA_TRAIN         * GA                + \
     B_BUSINESS_TRAIN   * BUSINESS          + \
     B_PRIVATE          * PRIVATE           + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                           + \
     B_COST          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2          + \
     B_AGE_ADULTS_SM * AGE_3          + \
     B_AGE_4_SM      * AGE_4          + \
     B_AGE_5_SM      * AGE_5          + \
     B_FIRST_SM      * FIRST          + \
     B_BUSINESS_SM   * BUSINESS       + \
     B_PRIVATE       * PRIVATE        + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                      + \
     B_COST     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED    

#Run MNL
LLNL3, nparamNL3, estimate_namesNL3, estimate_valuesNL3 = estimate_MNL(V1, V2, V3, model_name, 'NL', nest1, nest2)

In [73]:
likelihoodRatioTest(LL28, nparam28,'MNL28', LLNL3, nparamNL3,'MNLNL3', prob = 0.99)

LR = 0.00, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


# MNL (Allowing for non linear attributes)

In [0]:
%%capture
## Model 30
#Build on MNL28
#Generic non-linearization on cost
model_name = "MNL30"

#Utility functions
V1 = ASC_TRAIN                                      + \
     B_COST             * TRAIN_COST_SCALED**q_COST + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED           + \
     B_HE               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                                   + \
     B_COST          * SM_COST_SCALED**q_COST + \
     B_TIME_SM       * SM_TT_SCALED           + \
     B_HE            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                              + \
     B_COST     * CAR_COST_SCALED**q_COST + \
     B_TIME_CAR * CAR_TT_SCALED     

#Run MNL
LL30, nparam30, estimate_names30, estimate_values30 = estimate_MNL(V1, V2, V3, model_name)

In [75]:
likelihoodRatioTest(LL28, nparam28,'MNL28', LL30, nparam30,'MNL30', prob = 0.99)

LR = 136.76, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
## Model 31
#Generic non-linearization on time
model_name = "MNL31"

#Utility functions
V1 = ASC_TRAIN                                      + \
     B_COST             * TRAIN_COST_SCALED**q_COST + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED**q_TIME   + \
     B_HE               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                                   + \
     B_COST          * SM_COST_SCALED**q_COST + \
     B_TIME_SM       * SM_TT_SCALED**q_TIME   + \
     B_HE            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                              + \
     B_COST     * CAR_COST_SCALED**q_COST + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME     

#Run MNL
LL31, nparam31, estimate_names31, estimate_values31 = estimate_MNL(V1, V2, V3, model_name)

In [77]:
likelihoodRatioTest(LL30, nparam30,'MNL30', LL31, nparam31,'MNL31', prob = 0.99)

LR = 160.76, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [0]:
%%capture
# Model 32
#Generic non-linearization on HE
model_name = "MNL32"

#Utility functions
V1 = ASC_TRAIN                                            + \
     B_COST             * TRAIN_COST_SCALED**q_COST       + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED**q_TIME         + \
     B_HE               * TRAIN_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                           + \
     B_AGE_3_TRAIN      * AGE_3                           + \
     B_AGE_ADULTS_TRAIN * AGE_4                           + \
     B_AGE_5_TRAIN      * AGE_5                           + \
     B_LUGGAGE_TRAIN    * LUGGAGE                         + \
     B_MALE_TRAIN       * MALE                            + \
     B_GA_TRAIN         * GA                              + \
     B_BUSINESS_TRAIN   * BUSINESS                        + \
     B_PRIVATE          * PRIVATE                         + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                                      + \
     B_COST          * SM_COST_SCALED**q_COST    + \
     B_TIME_SM       * SM_TT_SCALED**q_TIME      + \
     B_HE            * SM_HE_SCALED**q_HE        + \
     B_AGE_ADULTS_SM * AGE_2                     + \
     B_AGE_ADULTS_SM * AGE_3                     + \
     B_AGE_4_SM      * AGE_4                     + \
     B_AGE_5_SM      * AGE_5                     + \
     B_FIRST_SM      * FIRST                     + \
     B_BUSINESS_SM   * BUSINESS                  + \
     B_PRIVATE       * PRIVATE                   + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                                + \
     B_COST     * CAR_COST_SCALED**q_COST   + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME     

#Run MNL
LL32, nparam32, estimate_names32, estimate_values32 = estimate_MNL(V1, V2, V3, model_name)

In [79]:
likelihoodRatioTest(LL31, nparam31,'MNL31', LL32, nparam32,'MNL32', prob = 0.99)

LR = 0.20, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


# Nested Logit with non-linear attributes

In [0]:
%%capture
## Model NL4
#Based on MNL 31
model_name = "NL4"

nest1 = [1,2] #TRAIN and SM
nest2 = [3]   #CAR

#Utility functions
V1 = ASC_TRAIN                                            + \
     B_COST             * TRAIN_COST_SCALED**q_COST       + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED**q_TIME         + \
     B_HE               * TRAIN_HE_SCALED                 + \
     B_AGE_ADULTS_TRAIN * AGE_2                           + \
     B_AGE_3_TRAIN      * AGE_3                           + \
     B_AGE_ADULTS_TRAIN * AGE_4                           + \
     B_AGE_5_TRAIN      * AGE_5                           + \
     B_LUGGAGE_TRAIN    * LUGGAGE                         + \
     B_MALE_TRAIN       * MALE                            + \
     B_GA_TRAIN         * GA                              + \
     B_BUSINESS_TRAIN   * BUSINESS                        + \
     B_PRIVATE          * PRIVATE                         + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                                      + \
     B_COST          * SM_COST_SCALED**q_COST    + \
     B_TIME_SM       * SM_TT_SCALED**q_TIME      + \
     B_HE            * SM_HE_SCALED              + \
     B_AGE_ADULTS_SM * AGE_2                     + \
     B_AGE_ADULTS_SM * AGE_3                     + \
     B_AGE_4_SM      * AGE_4                     + \
     B_AGE_5_SM      * AGE_5                     + \
     B_FIRST_SM      * FIRST                     + \
     B_BUSINESS_SM   * BUSINESS                  + \
     B_PRIVATE       * PRIVATE                   + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                                + \
     B_COST     * CAR_COST_SCALED**q_COST   + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME   

#Run MNL
LLNL4, nparamNL4, estimate_namesNL4, estimate_valuesNL4 = estimate_MNL(V1, V2, V3, model_name, 'NL', nest1, nest2)

In [81]:
likelihoodRatioTest(LL31, nparam31,'MNL31', LLNL4, nparamNL4,'NL4', prob = 0.99)

LR = 0.00, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


In [0]:
%%capture
## Model NL5
#Based on MNL 31
model_name = "NL5"

nest1 = [1,3] #TRAIN and CAR
nest2 = [2]   #SM

#Utility functions
V1 = ASC_TRAIN                                            + \
     B_COST             * TRAIN_COST_SCALED**q_COST       + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED**q_TIME         + \
     B_HE               * TRAIN_HE_SCALED                 + \
     B_AGE_ADULTS_TRAIN * AGE_2                           + \
     B_AGE_3_TRAIN      * AGE_3                           + \
     B_AGE_ADULTS_TRAIN * AGE_4                           + \
     B_AGE_5_TRAIN      * AGE_5                           + \
     B_LUGGAGE_TRAIN    * LUGGAGE                         + \
     B_MALE_TRAIN       * MALE                            + \
     B_GA_TRAIN         * GA                              + \
     B_BUSINESS_TRAIN   * BUSINESS                        + \
     B_PRIVATE          * PRIVATE                         + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                                      + \
     B_COST          * SM_COST_SCALED**q_COST    + \
     B_TIME_SM       * SM_TT_SCALED**q_TIME      + \
     B_HE            * SM_HE_SCALED              + \
     B_AGE_ADULTS_SM * AGE_2                     + \
     B_AGE_ADULTS_SM * AGE_3                     + \
     B_AGE_4_SM      * AGE_4                     + \
     B_AGE_5_SM      * AGE_5                     + \
     B_FIRST_SM      * FIRST                     + \
     B_BUSINESS_SM   * BUSINESS                  + \
     B_PRIVATE       * PRIVATE                   + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                                + \
     B_COST     * CAR_COST_SCALED**q_COST   + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME  

#Run MNL
LLNL5, nparamNL5, estimate_namesNL5, estimate_valuesNL5 = estimate_MNL(V1, V2, V3, model_name, 'NL', nest1, nest2)

In [83]:
likelihoodRatioTest(LL31, nparam31,'MNL31', LLNL5, nparamNL5,'NL5', prob = 0.99)

LR = 40.62, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Accept more params


In [86]:
LLNL5

-7705.909667914766

In [0]:
%%capture
## Model NL6
#Based on MNL 31
model_name = "NL6"

nest1 = [1]   #TRAIN 
nest2 = [2,3] #SM and CAR

#Utility functions
V1 = ASC_TRAIN                                            + \
     B_COST             * TRAIN_COST_SCALED**q_COST       + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED**q_TIME         + \
     B_HE               * TRAIN_HE_SCALED                 + \
     B_AGE_ADULTS_TRAIN * AGE_2                           + \
     B_AGE_3_TRAIN      * AGE_3                           + \
     B_AGE_ADULTS_TRAIN * AGE_4                           + \
     B_AGE_5_TRAIN      * AGE_5                           + \
     B_LUGGAGE_TRAIN    * LUGGAGE                         + \
     B_MALE_TRAIN       * MALE                            + \
     B_GA_TRAIN         * GA                              + \
     B_BUSINESS_TRAIN   * BUSINESS                        + \
     B_PRIVATE          * PRIVATE                         + \
     B_COMMUTER         * COMMUTER    

V2 = ASC_SM                                      + \
     B_COST          * SM_COST_SCALED**q_COST    + \
     B_TIME_SM       * SM_TT_SCALED**q_TIME      + \
     B_HE            * SM_HE_SCALED              + \
     B_AGE_ADULTS_SM * AGE_2                     + \
     B_AGE_ADULTS_SM * AGE_3                     + \
     B_AGE_4_SM      * AGE_4                     + \
     B_AGE_5_SM      * AGE_5                     + \
     B_FIRST_SM      * FIRST                     + \
     B_BUSINESS_SM   * BUSINESS                  + \
     B_PRIVATE       * PRIVATE                   + \
     B_COMMUTER      * COMMUTER     

V3 = ASC_CAR                                + \
     B_COST     * CAR_COST_SCALED**q_COST   + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME   

#Run MNL
LLNL6, nparamNL6, estimate_namesNL6, estimate_valuesNL6 = estimate_MNL(V1, V2, V3, model_name, 'NL', nest1, nest2)

In [88]:
likelihoodRatioTest(LL31, nparam31,'MNL31', LLNL6, nparamNL6,'NL6', prob = 0.99)

LR = 0.00, Chi2 = 6.63 (1 defrees of freedom) 
Therefore: Do not accept more params


NOTES TO ABOVE FUNCTION: 
- Right now i specify the utility functions manually. I want to have these as input (so we have one function for whatever model we wish), but I cant see how we can then change the attributes in the utilities.

Apart from that the function does what i want (There are definetly stuff that can be done to make it prettier and more clever, but I don't see anything major apart from the above point)